<h1 style="color:blue"> Bayesian Statistics and Probabilistic Programming</h1>

<h1 style="color:blue"> Spring 2024 - Homework 01</h1>

<h3 style="color:blue">2024-03-25</h3>

<h1 style="color:blue">Contributors</h1>

### Adriana Alvaro Email:
### Madison Chester Email:
### Dafni Tziakouri Email: dtziaktz7@alumnes.ub.edu
### Arturo Fredes Email:

<h1 style="color:blue">Exercises</h1>

## Exercise 1: Placenta previa

This exercise pertains to what we studied in `Binomial.3.Placenta.previa.pub.ipynb`.

1. Compute the posterior pdf, and both prior and posterior predictor pmf for the placenta previa problem, using a conjugate $\textrm{Beta}(\alpha,\beta)$ prior, for several values of the parameters, obtaining for each of them a 95% (centered) credible interval and the MAP and posterior expectation estimators of $\theta$. In particular, reproduce Figure 2.3 and Table 2.1 in BDA3 [1]. Do the same for those $\alpha_0$ and $\beta_0$ such that $\textrm{Beta}(\alpha_0,\beta_0)$ has the same expectation and variance than the piecewise linear nonconjugate prior. Compare the posterior quantities with both priors. 

2. Simulate the prior predictive pmf with the nonconjugate piecewise linear pdf. Do it in both possible ways: (1) Sample from $h$ by the inverse transformation method and (2) Sample from its discretization. Summarize the resulting simulated sample: plot the empirical pmf, compute median, mode, expectation, standard deviation. Compare the results with those obtained from the conjugate prior $\textrm{Beta}(\alpha_0,\beta_0)$. 

3. Simulate the posterior predictive pmf (given the observed data of 437 female births from 980 placenta previa births) with the nonconjugate piecewise linear pdf. Sample from the the discretization of the posterior pdf as in Exercise 1. Summarize the resulting simulated sample: plot the empirical pmf, compute median, mode, expectation, standard deviation. Compare the results with those obtained from the conjugate prior $\textrm{Beta}(\alpha_0,\beta_0)$.

[1] _Bayesian Data Analysis - Third edition_ **(BDA3),** pag. 37.

### Exercise 1.1

In [13]:
install.packages("ggplot2")
install.packages("BayesianFirstAid")

also installing the dependencies 'cli', 'gtable', 'lifecycle', 'rlang', 'scales', 'vctrs'

Warning message:
"unable to access index for repository https://cloud.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cloud.r-project.org/bin/windows/contrib/3.6/PACKAGES'"Packages which are only available in source form, and may need
  compilation of C/C++/Fortran: 'cli' 'rlang' 'scales' 'vctrs'


  These will not be installed


installing the source packages 'gtable', 'lifecycle', 'ggplot2'

Warning message in install.packages("ggplot2"):
"installation of package 'lifecycle' had non-zero exit status"Warning message in install.packages("ggplot2"):
"installation of package 'gtable' had non-zero exit status"Warning message in install.packages("ggplot2"):
"installation of package 'ggplot2' had non-zero exit status"Warning message:
"package 'BayesianFirstAid' is not available (for R version 3.6.1)"Warning message:
"unable to access index for repository https://cloud.r-project.org/bin/windows/contrib/3.6:
  cannot open URL 'https://cloud.r-project.org/bin/windows/contrib/3.6/PACKAGES'"

In [5]:
# Required Libraries
library(ggplot2)
library(BayesianFirstAid)

# Observed data
n <- 980
x <- 437

# Prior parameters for Beta distribution
alphas <- c(1, 5, 10, 20)
betas <- c(1, 5, 10, 20)

ERROR: Error in library(ggplot2): there is no package called 'ggplot2'


In [ ]:
# Function to compute posterior parameters and statistics
compute_posterior <- function(alpha, beta, x, n) {
  # Compute posterior parameters
  alpha_post <- alpha + x
  beta_post <- beta + n - x
  
  # Compute posterior statistics
  theta_mode <- (alpha_post - 1) / (alpha_post + beta_post - 2)
  theta_mean <- alpha_post / (alpha_post + beta_post)
  
  # Compute 95% centered credible interval
  cred_interval <- qbeta(c(0.025, 0.975), alpha_post, beta_post)
  
  # Return results
  return(list(alpha_post = alpha_post, beta_post = beta_post, 
              mode = theta_mode, mean = theta_mean,
              credible_interval = cred_interval))
}

# Compute posterior for different alpha and beta values
posterior_results <- lapply(alphas, function(alpha) {
  lapply(betas, function(beta) {
    compute_posterior(alpha, beta, x, n)
  })
})

In [ ]:
# Compare results with piecewise linear nonconjugate prior
# We'll use the results from the notebook for comparison

# Display posterior statistics for different priors
for (i in seq_along(alphas)) {
  for (j in seq_along(betas)) {
    cat(sprintf("Alpha = %d, Beta = %d:\n", alphas[i], betas[j]))
    cat("Conjugate Prior:\n")
    cat(sprintf("  Posterior Mode: %.3f\n", posterior_results[[i]][[j]]$mode))
    cat(sprintf("  Posterior Mean: %.3f\n", posterior_results[[i]][[j]]$mean))
    cat(sprintf("  95%% Credible Interval: [%.3f, %.3f]\n", 
                posterior_results[[i]][[j]]$credible_interval[1],
                posterior_results[[i]][[j]]$credible_interval[2]))
    cat("\n")
  }
}

### Exercise 1.2

### Exercise 1.3

## Exercise 2: Asthma mortality

This exercise pertains to `Poisson.01.pub.ipynb` (lecture 7). Review this notebook as well as the notes concerning the Poisson model and do the following:

 1. Generate a sample from the prior predictive distribution.
 
 2. Generate a sample from the posterior predictive distribution after the first experiment (with $y=3$ after one year in a population of 200,000). 

In both cases plot the simulated pmf and compare it with the theoretical one and estimate the probability of having more than 25 fatalities in the next 10 years (assuming all other conditions remain constant).